# The Heston Model and Forecasting Option Market Values

**2025 Introduction to Quantiative Methods in Finance**

**The Erdös Institute**

In [ ]:
#Package Import
import numpy as np
import pandas as pd
import yfinance as yf
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm
from scipy.stats import shapiro
import scipy.stats as stats
from scipy.optimize import brentq
from scipy.integrate import quad
sns.set_style('darkgrid')
from scipy.optimize import minimize
from joblib import Parallel, delayed
from scipy.stats import rv_discrete
from statsmodels.stats.diagnostic import acorr_ljungbox


%run functions.py


### The $n$-step discrete Heston Model:

Let $0 = t_0<t_1<\cdots <t_n = t$, then the $n$-step discrete Heston model assumes for each $1\leq i\leq n$ the variance of the log-returns at time $t_i$ is modeled as

$$v_{t_i} = |v_{i-1} + \kappa(\theta - v_{t_{i-1}})(t_i-t_{i-1}) + \xi\sqrt{|v_{i-1}|(t_i-t_{i-1})}\mathcal{N}^{v}(0,1)|$$

and the distribution of stock paths from time $t_{i-1}$ to $t_i$ is modeled as

$$S_{t_i} = S_{t_{i-1}}e^{(\mu + r - .5v_{t_{i-1}})(t_i-t_{i-1}) + \sqrt{|v_{t_{i-1}}|(t_i-t_{i-1})}\mathcal{N}^S(0,1)}$$
where:

- $S_0$ is the initial stock price;
- $S_t$ is the stock price at time $t$;
- $v_0$ is the variance of the stock at time $0$;
- $v_t$ is the variance of the stock at time $t$;
- $\mu$ is the excess drift of the log-returns of the stock;
- $r$ is the risk-free interest rate;
- $\kappa$ is the **mean-reversion rate** of the variance process;
- $\theta$ is the **long-run variance level** of the variance process;
- $\xi$ is the **volatility of volatility**
- $\mathcal{N}^{v}(0,1)$ and $\mathcal{N}^{S}(0,1)$ are standard normal distributions with correlation $\rho$


### Remark 1:

The general definition of the Heston Model describes the variance and stock paths as being modeled as intertwined solutions to a system of stochastic Partial Differential Equations. The limiting distribution of variances and stock paths of the discrete models as the number of steps tends to $\infty$ is the continuous model described by the system of stochastic partial differential equations.

### Remark 2:

The Heston model is a generalization of the Black-Scholes/Geometric Brownian Motion Model. The two models agree if:
- $v_0 = \theta$
- $\kappa = 0$
- $\xi = 0$.

### Remark 3:

It is common practice to use $\mbox{max}(v_i,0)$ instead of $|v_i|$ in the Heston model.

In [ ]:
### Visual of stock paths measure simulated through the Heston model

kappa =  #Mean reversion rate of variance of log-returns (Typical range 1 to 5)

theta =  #Long run variance of log-returns

xi =  # Volatility of Volatility (Typical range .2 to 1)

v0 =  #Initial variance of log-returns

rho =  #Correlation of random noise of variance and log-returns (Typical range -.9 to -.4)

S0 =  #Initial Stock Value

t =  #Time to expiration

n_steps =  #Number of steps in each simulation

r =  #Risk-free interest rate

n_sims =  #Number of simulations

dt =  #Time between each step

mu =  #Excess drift of log returns



heston_paths = heston_path_sim(S0, v0, r, t, n_steps, kappa, theta, xi, rho, n_sims, mu)

plt.figure(figsize = (12,8))
for path in heston_paths:
    plt.plot(path)
    
    
plt.title('Heston Path Simulations', size = 20)

plt.show()

In [ ]:
#Use Monte-Carlo methods to simulate price of call option under Heston-Model.
#Compare estimate provided by faster method of numeric integration.





#Monte-Carlo Simulation
call_payouts_hedged = heston_call_MC(S0, K, v0, r, t, n_steps, kappa, theta, xi, rho, n_sims, mu)

heston_call_sim = np.mean(np.mean(call_payouts_hedged))

heston_std_err = np.std(call_payouts_hedged)/np.sqrt(n_sims)

heston_call_price = heston_call(S0, K, v0, r, t, kappa, theta, xi, rho)


print(f'Monte-Carlo Simulated value of Call Option, Heston Model: ${heston_call_sim:.2f}\
 with standard error {heston_std_err:.6f}.')


print('-----'*10)
print('-----'*10)


print(f'Actual value of call option under Heston model assumption: ${heston_call_price:.2f}.')


print('-----'*10)
print('-----'*10)


plt.figure(figsize = (8,6))

plt.hist(call_payouts_hedged, bins = 75, color = 'pink', alpha = .8, label = 'Simulated Heston Call Prices')
plt.axvline(heston_call_price, label = 'Heston Call Price',color = 'black', lw = '7', alpha = 1)
plt.axvline(heston_call_sim, label = 'Heston Call Simulated Price', color = 'red', ls = '--')



plt.title(f'Simulated Call Option Values, Heston Model with {n_steps} control variates', size = 18)



plt.legend()

plt.show()


In [ ]:
#Use Monte-Carlo methods to simulate price of call option under Heston-Model  
#with an increasing number of steps = delta-based control variates.
#Compare estimate provided by faster method of numeric integration.





steps = [1, 2, 3, 5, 10, 50, 252]

for n_steps in steps:


    #Monte-Carlo Simulation
    call_payouts_hedged = heston_call_MC(S0, K, v0, r, t, n_steps, kappa, theta, xi, rho, n_sims, mu=0)

    heston_call_sim = np.mean(np.mean(call_payouts_hedged))

    heston_std_err = np.std(call_payouts_hedged)/np.sqrt(n_sims)

    heston_call_price = heston_call(S0, K, v0, r, t, kappa, theta, xi, rho)


    print(f'Heston Monte-Carlo Simulated value of Call Option with {n_steps} control variates\
 and {n_sims} simulations: ${heston_call_sim:.2f} with standard error {heston_std_err:.6f}.')


    print('-----'*10)
    print('-----'*10)


    print(f'Actual value of call option under Heston model assumption: ${heston_call_price:.2f}.')


    print('-----'*10)
    print('-----'*10)


    plt.figure(figsize = (8,6))

    plt.hist(call_payouts_hedged, bins = 75, color = 'pink', alpha = .8, label = 'Simulated Heston Call Prices')
    plt.axvline(heston_call_price, label = 'Heston Call Price',color = 'black', lw = '7', alpha = 1)
    plt.axvline(heston_call_sim, label = 'Heston Call Simulated Price', color = 'red', ls = '--')



    plt.title(f'Simulated Call Option Values, Heston Model with {n_steps}\
 control variates and {n_sims} simulations', size = 15)



    plt.legend()

    plt.show()


In [ ]:
### The Heston model agrees with the Black-Scholes model if kappa and xi are 0 and v0 = theta.




call_payouts_hedged = heston_call_MC(S0, K, v0, r, t, n_steps, kappa, theta, xi, rho, n_sims, mu=0)

heston_call_sim = np.mean(np.mean(call_payouts_hedged))

heston_std_err = np.std(call_payouts_hedged)/np.sqrt(n_sims)

black_scholes_price = bs_call(S0,K,np.sqrt(theta),t,r)


print(f'Monte-Carlo Simulated value of Call Option, Heston Model: ${heston_call_sim:.2f}\
 with standard error {heston_std_err:.6f}.')


print('-----'*10)
print('-----'*10)




print(f'Value of call option under Black-Scholes assumption: ${black_scholes_price:.2f}.')


In [ ]:
### Methods of numeric integration allow for quick valuation of option values under Heston model

#Heston Parameters.



#Strike Price
K = 99

ttes = [.01, .2, .5, .75, 1, 1.5]
strikes = range(90, 111)


heston_prices = [[heston_call(S0, K, v0, r, t, kappa, theta, xi, rho) for K in strikes]for t in ttes]

price_matrix = pd.DataFrame(heston_prices, index = ttes, columns = strikes)
price_matrix.index.name = 'Time to Expiration (Years)'
price_matrix.columns.name = 'Strike Price'


price_matrix

### Historical Volatility Distributions

Historical volatilities of stock paths are more accurate captured by the Heston model over the Black-Scholes model.

Through hedging, the value of an option is a measurement of the behavior of volatility of the stock. Meaning that option values are neutral to stock drift with strategically implemented hedging strategies.

Therefore the Heston model has the potential to more accurately capture volatility behavior of a stock path and therefore provide more reliable measurements of an option contract's value.

In [ ]:
#Visual of simulated historical volatilities realized under Black-Scholes assumptions

### Simulated stock paths under Geometric Brownian Motion (Black-Scholes Model)
S0 = 
sigma = 
t = 
r = 
mu = 
n_steps = 
n_sims = 



paths = gbm_path_sim(S0, sigma, t, r, mu, n_sims, n_steps)

#Convert simulated paths to pandas series
paths = [pd.Series(path) for path in paths]


# Compute 21-day volatilities of paths
path_rolling_vols = [np.log(path / path.shift(1)).rolling(window=21).std() * np.sqrt(252) for path in paths]

#Concatenate the rolling volatilities of the simulated paths
all_rolling_vols = pd.concat(path_rolling_vols, ignore_index=True).dropna()

#Create Histogram of volatilities

plt.figure(figsize = (12,8))
plt.hist(all_rolling_vols, bins = 70, alpha = .6, label = 'Historical Volatilities')
plt.axvline(np.mean(all_rolling_vols), color = 'red', label = 'Mean Historical Volatility')
plt.legend()

plt.title('Simulated Historical Volatilities of Geometric Brownian Motion simulated stock paths', size = 15)

plt.show()

In [ ]:
#Real World Distribution of Historical volatilites of a stock

tickers = []

for ticker in tickers:

    stock = 

    #Historical volatilities of stock measured over 21 day trading days
    historical_vols = 

    plt.figure(figsize = (9,6))
    plt.hist(historical_vols, bins = 70, color = 'red', alpha = .7)
    plt.title(f'Historical volatilities of {ticker}', size = 15)
    plt.show()

In [ ]:
### Simulated Historical volatilites of stock path under Heston model.


#Heston Parameters.




paths = heston_path_sim(S0, v0, r, t, n_steps, kappa, theta, xi, rho, n_sims, mu, False)



#Convert simulated paths to pandas series
paths = [pd.Series(path) for path in paths]


# Compute 21-day volatilities of paths
path_rolling_vols = 

#Concatenate the rolling volatilities of the simulated paths
all_rolling_vols = pd.concat(path_rolling_vols, ignore_index=True).dropna()

#Create Histogram of volatilities

plt.figure(figsize = (12,8))
plt.hist(all_rolling_vols, bins = 70, alpha = .6, label = 'Historical Volatilities')
plt.axvline(np.mean(all_rolling_vols), color = 'red', label = 'Mean Historical Volatility')

plt.title('Simulated Historical Volatilities of Heston simulated stock paths', size = 15)

plt.legend()

plt.show()

### Question

Can the distribution of historical volatilites be used to create a discrete distribution that successefully models the call value of the Heston model?


If the answer is yes, then simulation of the Heston model can be dramatically sped up.


Below we simulate that a discrete distribution does not capture the Heston model.

In [ ]:
#From a discrete distribution of volatilities from simulated historical volatilities under Heston model
counts, bin_edges = np.histogram(all_rolling_vols, bins=50)
bin_midpoints = (bin_edges[:-1] + bin_edges[1:]) / 2
probabilities = counts / counts.sum()
discrete_dist = rv_discrete(name='hist_based', values=(bin_midpoints, probabilities))

#Heston Parameters.



# Simulate volatility paths
noise = discrete_dist.rvs(size=(n_sims, n_steps))
log_returns = (r - 0.5 * noise**2) * dt + noise * np.sqrt(dt)
log_price_paths = np.cumsum(log_returns, axis=1)
paths = S0 * np.exp(log_price_paths)
paths = np.hstack([S0 * np.ones((n_sims, 1)), paths])

# Delta hedging
deltas = np.array([
    bs_call_delta(paths[:, i], K, noise[:, i], t - i * dt, r)
    for i in range(n_steps)
]).T

stock_profits_steps = (paths[:, 1:]-np.exp(r * dt)*paths[:, :-1])*deltas*np.exp(-r*np.arange(1,n_steps+1)*dt)

stock_profits = np.sum(stock_profits_steps, axis=1)

# Payoffs
call_payoffs = np.maximum(paths[:, -1] - K, 0) * np.exp(-r * t)
sim_call_values = call_payoffs - stock_profits

# Output
sim_call_value = np.mean(sim_call_values)
std_err = np.std(sim_call_values) / np.sqrt(n_sims)
heston_call_price = heston_call(S0, K, v0, r, t, kappa, theta, xi, rho)

print(f'Simulated call value from discrete distribution: ${sim_call_value:.2f} with standard error {std_err:.6f}')
print('--------' * 10)
print('--------' * 10)
print(f'Heston value of call option: ${heston_call_price:.2f}.')
print('--------' * 10)
print('--------' * 10)
print(f'Discrete distributions do not capture autoregressive influence of variance found of the Heston model.')

In [ ]:
#Statistical test if there is autoregressive behavior in historical volatilities
tickers = 
for ticker in tickers:
    stock = yf.download(ticker, period = '15y', interval = '1d')
    #Historical volatilities of stock measured over 21 day trading days
    historical_vols = np.log(stock['Close']/stock['Close'].shift(1)).rolling(window = 21).std()*np.sqrt(252)
    data = np.asarray(historical_vols.dropna())
    ljung_result = acorr_ljungbox(data, lags=[10], return_df=True)
    p_value = ljung_result['lb_pvalue'].iloc[0]
    if p_value < 0.05:
        print(f'p-value: {p_value} - Reject null hypothesis: statistical evidence of autoregressive \
behavior in historical volatility of {ticker}.')
        print('--------'*10)
        print('--------'*10)
    else:
        print(f'p-value: {p_value} - Not statistically significant to reject null hypothesis: \
no clear autoregressive behavior in historical volatility of {ticker}.')
        print('--------'*10)
        print('--------'*10)

### Heston Calibration using Option Data

Standard methods of minimizing errors measured in terms of sums of least squares of the implied volatilities of option market price data is a standard method of estimating parameters of Heston model to forecast future distribution of option prices on the market.

In [ ]:
##yfinance does provide enough option data to play around with Heston calibration
stock_symbol = 'AAPL'
ticker = yf.Ticker(stock_symbol)
stock_data = yf.download(stock_symbol, period = '1d', interval = '1m')



expirations = ticker.options
# Store results
option_data = []

for date in expirations:
    chain = ticker.option_chain(date)

    # Add expiration and label
    calls = chain.calls.copy()
    calls['expiration'] = date
    calls['option_type'] = 'call'

    puts = chain.puts.copy()
    puts['expiration'] = date
    puts['option_type'] = 'put'

    option_data.append(calls)
    option_data.append(puts)

# Combine all into one DataFrame
options_data = pd.concat(option_data, ignore_index=True)

In [ ]:
options_data

In [ ]:
##Functions for option data obtained through y-finance

from dateutil import parser
from dateutil.tz import tzutc

def find_tte_yf_options(expiration_date,last_trade_date):
    '''returns time measured in years as a float between two dates
    
    Inputs:
    expiration_date (str): 'YYYY-MM-DD'
    last_trade_date (pandas._libs.tslibs.timestamps.Timestamp)
    
    Returns:
    Float of time to expiration in years
    '''
    tte = (datetime.datetime.strptime(expiration_date+'-21-30', "%Y-%m-%d-%H-%M").replace(tzinfo=tzutc()) -\
last_trade_date).total_seconds()/(60*60*24*365)
    
    return tte


def yf_find_approx_spot(stock_data, last_trade_date):
    """
    Finds approximate spot price at the time of last trade.
    The spot price is approximate since yfinance does not provide 1-second data, only minute-by-minute.

    Parameters:
    stock_data (pd.Series or pd.DataFrame): stock prices with DatetimeIndex
    last_trade_date (pd.Timestamp): timestamp of last trade of option contract

    Returns:
    float: approximate spot price, or NaN if unavailable
    """
    # Round timestamp to the minute (zero out seconds)
    ts = last_trade_date.replace(second=0)

    try:
        return stock_data.loc[ts].iloc[0]
    except KeyError:
        return float('nan')


In [ ]:
stock_symbol = 'AAPL'
ticker = yf.Ticker(stock_symbol)
stock_data = yf.download(stock_symbol, period = '1d', interval = '1m')



expirations = ticker.options


#Create array to store options data
option_data = []

for date in expirations:
    chain = ticker.option_chain(date)

    # Add expiration and label
    calls = chain.calls.copy()
    calls['expiration'] = date
    calls['option_type'] = 'call'

    puts = chain.puts.copy()
    puts['expiration'] = date
    puts['option_type'] = 'put'

    option_data.append(calls)
    option_data.append(puts)

# Combine all into one DataFrame and delete implied volatility column since we'll recalculate.
options_data = pd.concat(option_data, ignore_index=True)
options_data = options_data.drop(columns=['impliedVolatility'])


#Delete any options that were traded in the interval of historical stock values obtained
start_date = stock_data.index[0]
options_data = options_data[options_data['lastTradeDate']>=start_date]


#Insert column of time to expiration in years of the option contract measured from time of last trade
options_data['time_to_expiration'] = options_data.apply(
lambda row: find_tte_yf_options(expiration_date = row['expiration'],
                               last_trade_date = row['lastTradeDate']),
    axis = 1
)


#Add in column of the spot price of stock when the option trade occured.
options_data['spot_price'] = options_data.apply(
    lambda row: yf_find_approx_spot(stock_data['Close'], row['lastTradeDate']),
    axis=1
)
options_data = options_data.dropna()





#Create data frames that keeps relevant information and separate calls from puts.
options_data = options_data[['strike', 'lastPrice', 'lastTradeDate',\
                             'expiration', 'option_type','time_to_expiration', 'spot_price']]
options_data_calls = options_data[(options_data['option_type'] == 'call')].copy()
options_data_puts = options_data[(options_data['option_type'] == 'put')].copy()



#Add implied volatility column in calls data frame 
r = 0.039
options_data_calls['implied_volatility'] = options_data_calls.apply(
    lambda row: implied_volatility_call(
        market_price=row['lastPrice'],
        S0 = row['spot_price'],
        K=row['strike'],
        t=row['time_to_expiration'],
        r=r
    ),
    axis=1
)


#Add implied volatility column in puts data frame 
options_data_puts['implied_volatility'] = options_data_puts.apply(
    lambda row: implied_volatility_put(
        market_price=row['lastPrice'],
        S0 = row['spot_price'],
        K=row['strike'],
        t=row['time_to_expiration'],
        r=r
    ),
    axis=1
)

#Remove rows with undefined values
options_data_calls = options_data_calls.dropna()
options_data_puts = options_data_puts.dropna()

In [ ]:
##Examine call options data
options_data_calls

In [ ]:
## Remove additional rows if needed
options_data_calls = options_data_calls[(options_data_calls['time_to_expiration']<=1)\
                                        & (options_data_calls['time_to_expiration']>=.5)]

In [ ]:
#Visual of volatility smiles from option data

for date in np.unique(options_data_calls['expiration'].values):
    options_with_same_expiration =  options_data_calls[options_data_calls['expiration']==date].copy()
    tte = options_with_same_expiration['time_to_expiration'].iloc[0]
    plt.figure(figsize = (10,8))
    plt.plot(options_with_same_expiration['strike'], options_with_same_expiration['implied_volatility'])

    plt.title(f'Market Volatility Smile Options with time to expiration: {tte:4f} years', size = 18)

    plt.ylabel('Implied Volatility', size = 15)
    plt.xlabel('Strike', size = 15)


    plt.show()

In [ ]:
###Calibrate Heston model to option data using gradient descent.

def objective(params, option_data, r):
    kappa, theta, xi, rho, v0 = params
    
    error = 0
    for _, row in option_data.iterrows():
        model_price = heston_call(
            S0=row['spot_price'],
            K=row['strike'],
            v0=v0,
            r=r,
            t=row['time_to_expiration'],
            kappa=kappa,
            theta=theta,
            xi=xi,
            rho=rho
        )
        market_price = row['lastPrice']
        error += (model_price - market_price) ** 2
        
    return error / len(option_data)


initial_guess = [1.0, 0.04, 0.3, -0.5, 0.04]  # kappa, theta, xi, rho, v0
bounds = [(1e-4, 10), (1e-4, 1), (1e-4, 2), (-0.99, 0.99), (1e-4, 1)] 


r = 0.039  

result = minimize(
    objective,
    initial_guess,
    args=(options_data_calls, r),
    bounds=bounds,
    method='L-BFGS-B',
    options={
        'disp': True,
        'maxiter': 20,       
        'ftol': 1e-2,         
        'gtol': 1e-2          
    }
)


calibrated_params = result.x

In [ ]:
print(f"Calibrated parameters:\
kappa = {calibrated_params[0]}, \
theta = {calibrated_params[1]}, \
xi={calibrated_params[2]}, \
rho = {calibrated_params[3]}, \
v0 = {calibrated_params[4]}")

### Quicker calibration techniques

The above calibrates all parameters of the Heston model. 

The process can be quickened if you have specific values for some of the parameters. For example, initial variance $v_0$ and long-run variance $\theta$ can be obtained via measurements of historical data.

The process can also be accelerated by relaxing the numerical integration accuracy in the heston call function.

In [ ]:
#Create a distribution of the value of 
#hedging a call option to a future date that is before the expiration of the call.


# Simulation
S0 = stock_data['Close'].iloc[-1].iloc[0]
kappa, theta, xi, rho, v0 = calibrated_params
K = 
tte = 

n_steps = 100*2
n_sims = 3000
t = 100 / 252

paths, vols = heston_path_sim(S0, v0, r, t, n_steps, kappa, theta, xi, rho, n_sims, 0, return_vol=True)

# Delta computation via Black-Scholes
dt = t / n_steps
deltas = np.array([
    bs_call_delta(paths[:, i], K, np.sqrt(np.maximum(vols[:, i], 1e-10)), tte - i * dt, r)
    for i in range(n_steps)
]).T

# Stock hedging P&L
stock_profits_steps=(paths[:,1:]-np.exp(r*dt)*paths[:,:-1])*deltas*np.exp(-r*np.arange(1,n_steps+1)*dt)
stock_profits = np.sum(stock_profits_steps, axis=1)

# Heston call value at horizon
future_tte = tte - t
terminal_prices = paths[:, -1]
terminal_vols = vols[:, -1]

call_prices = Parallel(n_jobs=-1)(
    delayed(heston_call)(S, K, v, r, future_tte, kappa, theta, xi, rho)
    for S, v in zip(terminal_prices, terminal_vols)
)

simulated_call_market_value = np.exp(-r*t)*np.array(call_prices)-stock_profits

avg_value = np.mean(simulated_call_market_value)

# Plot
plt.figure(figsize=(8, 6))
plt.hist(simulated_call_market_value, bins=60, color = 'red', alpha = .4, label='Simulated distribution of call option value')
plt.title('Simulated Distribution of Call Option Value with hedging before expiration', size=15)
plt.xlabel('Forecasted call value with hedging')
plt.axvline(avg_value, label = f'Mean value: ${avg_value: .2f}')
plt.legend()
plt.show()